# Synthetic Cluster Generation Tutorial Part 3: random realizations

Owner: **Tamas Norbert Varga** @vargatn

This notebook relies on one external package in addition to the default desc kernels on NERSC


This notebook will describe how to perform random draws of galaxy clusters from the statistical model trained in Part 2 of the tutorial


**Objectives:**

    * Load and collate galaxy catalogs 
    * Set up cluster line-of-sight emulation script
    * INTENSIVE: Draw samples from the proposal galaxy catalog and calculate survival scores for rejection sampling.
    * Define KDE model for cluster member galaxy features
    * Visualize model against simulations

Logistics: This notebook is intended to be be run with a python 3 installation of the skysampler package https://github.com/vargatn/skysampler/tree/lsst-dev  

Other notes:

    Some calculations in this notebook are time consuming, and are optimized for a HPC environment other than NERSC. These cells are currently commented out.
    
Output:

    The results of this notebook are made available on NERSC at /global/cscratch1/sd/tvarga/DC_DATA/


In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp
import skysampler.emulator as emulator
import skysampler.indexer as indexer

import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import skysampler.utils as utils


config read from file: /home/moon/vargatn/DES/Y3_WORK/skysampler-config/config/desy3-gamma-wide-config.yaml


In [2]:
def result_reader2(samples, scores, m_factor=20, nratio=1., seed=None):
    
    dc_score = np.exp(scores["dc"]) * np.abs(scores["dc_jac"])
    wr_score = np.exp(scores["wr"]) * np.abs(scores["wr_jac"])
    wcr_clust_score = np.exp(scores["wcr_clust"]) * np.abs(scores["wcr_clust_jac"])
    wcr_rands_score = np.exp(scores["wcr_rands"]) * np.abs(scores["wcr_rands_jac"])
#     fcls = fclfunc(samples["LOGR"]) - 1.
    
#     umult = np.max((fcls, np.ones(len(fcls))), axis=0)
#     print(umult)
    rng = np.random.RandomState(seed)
    uniform = rng.uniform(0, 1, len(samples)) #* umult
    
    p_proposal = m_factor * dc_score * wr_score
    p_rands_ref = wcr_rands_score
    p_clust_ref = wcr_clust_score #* fcls
    print(nratio)
    inds_field = (uniform < (p_rands_ref / nratio / p_proposal))
    inds_clust = ((p_rands_ref / nratio / p_proposal) < uniform) * (uniform < (p_clust_ref  / p_proposal))
    inds_2d = ((uniform < (p_clust_ref  / p_proposal)))

    return inds_field, inds_clust, inds_2d

In [6]:
tag_root = "dc2-alpha_concentric_sample-v01_test-03"
NREPEATS = 4
NSAMPLES = 1600000
NCHUNKS = 160
bandwidth=0.1

root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/"
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

LOGR_DRAW_RMINS = np.array([-3, -0.5, 0., 0.5])
LOGR_DRAW_RMAXS = np.array([-0.5, 0., 0.5, 1.2])
LOGR_CAT_RMAXS = [0., 0.5, 1.1, 1.2]

deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}

columns = {
    "cols_dc": ["COLOR_G_R", "COLOR_R_I",],
    "cols_wr": ["LOGR",],
    "cols_wcr": ["COLOR_G_R", "COLOR_R_I", "LOGR",],
}

In [3]:
deep_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

In [4]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

Note: detected 160 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
Note: NumExpr detected 160 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [7]:

tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading deep catalogs
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)

(1513572, 9)
(1513572,)
(1513572, 8)
(1513572,)
(1009658, 9)
(1009658,)


,GABS,SIZE,MAG_I,COLOR_G_R,COLOR_R_I,COLOR_I_Z,STELLAR_MASS,HALO_MASS,LOGR
0,0.423296,-0.998081,23.432575,1.035782,0.635422,0.108282,8.996163,11.038178,1.082454
1,0.064420,-0.608224,23.397775,0.549423,0.592930,0.134798,8.928065,11.388560,0.944479
2,0.272016,-0.778048,24.850255,0.030144,-0.021320,-0.100122,9.940126,11.850001,1.155868
3,0.239932,-0.669584,24.946480,0.820631,0.313587,0.142025,7.858064,10.906794,1.180757
4,0.088235,-1.112695,24.913888,0.695768,0.768230,0.748838,8.591421,10.846796,1.186534
...,...,...,...,...,...,...,...,...,...
1009653,0.204964,-0.247260,21.238839,1.374701,1.163914,0.457443,10.620822,12.249214,0.699411
1009654,0.402510,-0.625730,24.658276,0.592995,0.836044,0.101049,8.180118,11.682604,0.697040
1009655,0.088761,-0.255587,22.433150,0.224419,0.601627,0.055916,9.495716,13.134520,1.065588
1009656,0.401077,-0.531469,23.525604,0.471897,0.248249,0.151350,7.063112,10.373028,0.935907


In [8]:
samples = []
scores = []
for rbin in np.arange(4):
    print(rbin)
#     expr = "/e/ocean1/users/vargatn/EMULATOR/EPSILON/resamples/epsilon_concentric_sample_v06_run*_rbin" + str(rbin) + "*samples.fits" 
    expr = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/dc2-alpha_concentric_sample-v01_test-03/dc2-alpha_concentric_sample-v01_test-03_run0*_rbin" + str(rbin) + "*samples.fits" 

    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    samples.append(np.hstack(samples_sep))
    scores.append(np.hstack(scores_sep))

0
1
2
3


In [11]:
mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])

In [12]:
magcol = "mag_i"
ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = clust_los_nums / rands_los_nums

/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [13]:
ifields2 = []
iclusts2 = []
i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = result_reader2(samples[rbin], scores[rbin], nratio=nratios[rbin], m_factor=100, seed=rbin)
    ifields2.append(_ifield)
    iclusts2.append(_iclust)
    i2ds2.append(_i2d)

0
2.741989144315195
1
1.7913041757345514
2
1.1243442810539395
3
0.793271619957704


In [14]:
csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _csample = kde.random_draw(4e6)
    csamples.append(_csample)


0
1
2
3


In [15]:
rsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][ifields2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _rsample = kde.random_draw(4e6)
    rsamples.append(_rsample)

0
1
2
3


In [16]:
allsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][i2ds2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _sample = kde.random_draw(4e6)
    allsamples.append(_sample)

0
1
2
3


In [18]:
bcgtab = pd.DataFrame()
bcgtab["RA"] = [0,]
bcgtab["DEC"] = [0.,]
bcgtab["X"] = [0.,]
bcgtab["Y"] = [0.,]

_imag = 17.62
_gr = 1.38
_ri = 0.54
_iz = 0.37
bcgtab["MAG_G"] = [_gr + _ri + _imag,]
bcgtab["MAG_R"] = [_ri + _imag,]
bcgtab["MAG_I"] = [_imag,]
bcgtab["MAG_Z"] = [_imag - _iz,]
bcgtab["FRACDEV"] = [1.,]
bcgtab["TSIZE"] = [33.20,]

bcg_gabs = 0.14
# bcg_gabs = 0.07
angle = np.random.uniform(0, np.pi, size=1)

bcgtab["G1"] = -1. * bcg_gabs * np.cos(2 * angle)
bcgtab["G2"] = -1. * bcg_gabs * np.sin(2 * angle)
# bcgtab["G1"] = [0,]
# bcgtab["G2"] = [0,]
bcgtab["LOC"] = 1

bcgtab["Z"] = 0.325
bcgtab["shear"] = 0.
# bcgtab["shear2"] = 0.
# bcgtab["Z"]

bcgtab["FLUX_G"] = 10 ** (0.4 *(30. - bcgtab["MAG_G"]))
bcgtab["FLUX_R"] = 10 ** (0.4 *(30. - bcgtab["MAG_R"]))
bcgtab["FLUX_I"] = 10 ** (0.4 *(30. - bcgtab["MAG_I"]))
bcgtab["FLUX_Z"] = 10 ** (0.4 *(30. - bcgtab["MAG_Z"]))

In [26]:
ctab = _wide_cr_settings_clust["container"].data
rtab = _wide_cr_settings_rands["container"].data

In [64]:
magcol = "mag_i"
magcol_sample = "MAG_I"
rcol = "LOGR"
mag_lims = (17, 22.5)
medges = np.linspace(17, 22.5, 6)
medges2 = np.linspace(22.5, 24.5, 4)

mag_slices = [(17, 19), (19, 20), (20, 21), (21, 22), (22, 22.5)]

r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])


ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = np.max((clust_los_nums / rands_los_nums, np.ones(4)), axis=0)
# TODO there is a footprint masking bug which is fixed in the above line

frac = 1. / (clust_los_nums / rands_los_nums)
fac = 1
clust_nums_to_draw = clust_los_nums * (1 - frac) * fac
rands_nums_to_draw = rands_los_nums * fac


maglim_max1 = 22.5
maglim_max2 = 24.5
clust_gals = []
field_gals = []

# Becouse of the above surface density bug, we restrict the radial range to the inner 3 radial ranges
for rbin, r_lims in enumerate(r_lims_all[:3]):
    print(rbin, r_lims)

    #######################################
    sample = csamples[rbin]

    ii = ((np.log10(table["R"]) > r_lims[0]) & (np.log10(table["R"]) < r_lims[1]))
    vals = np.histogram(table[ii][magcol], bins=medges)[0]
    vals = vals / vals.sum()

    for i in np.arange(len(medges) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i]
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol_sample] > medges[i]) & (sample[magcol_sample] < medges[i + 1]) &
                 (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
        ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
        tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   

    ii = ((sample[magcol_sample] > mag_lims[0]) & (sample[magcol_sample] < maglim_max1) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num1 = len(sample[ii])
# 
    ii = ((sample[magcol_sample] > maglim_max1) & (sample[magcol_sample] < maglim_max2) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num2 = len(sample[ii])
    fac_clust = num2 / num1

    vals = np.histogram(sample[magcol_sample], bins=medges2)[0]
    vals = vals / vals.sum()
    for i in np.arange(len(medges2) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i] * fac_clust
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol_sample] > medges2[i]) & (sample[magcol_sample] < medges2[i + 1]) &
        (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
#         ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
#         tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   



gals = pd.concat(clust_gals, sort=True)
gals["Z"] = 0.325


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


0 (-1.5, -0.5)


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


1 (-0.5, 0.0)


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


2 (0, 0.5)


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [66]:
_imag = 17.62
_gr = 1.38
_ri = 0.54
_iz = 0.37
bcg_table = pd.DataFrame()
bcg_table["raJ2000"] = np.deg2rad(np.array([64,]))
bcg_table["decJ2000"] = np.deg2rad(np.array([-36,]))
bcg_table['sourceType']  = np.array(["galaxy",])
bcg_table['DiskHalfLightRadius'] = np.array([5.76,])
bcg_table['BulgeHalfLightRadius'] = np.array([5.76])
bcg_table['gmagVar'] = np.array([_gr + _ri + _imag,])
bcg_table['rmagVar'] = np.array([_ri + _imag,])
bcg_table['imagVar'] = np.array([_imag,])
bcg_table['zmagVar'] = np.array([_imag - _iz,])
bcg_table['disk_n'] = np.array([1.0,])
bcg_table['bulge_n'] = np.array([3.0,])
bcg_table['a_d'] = np.array([2.0,])
bcg_table['a_b'] = np.array([2.0,])
bcg_table['b_d'] = np.array([2.0,])
bcg_table['b_b'] = np.array([2.0,])
bcg_table['pa_disk'] = np.array([35.0,])
bcg_table['pa_bulge'] = np.array([35.0,])

np.random.seed(10)
angle = np.random.uniform(0, 2. * np.pi, size=len(gals))
table = pd.DataFrame()
table["raJ2000"] = np.deg2rad((10**gals["LOGR"] * np.cos(angle)).values / 60. + 64)
table["decJ2000"] = np.deg2rad((10**gals["LOGR"] * np.sin(angle)).values / 60. - 36)
table['sourceType'] = np.array(["galaxy",]*len(gals))
table['DiskHalfLightRadius'] = 10**gals["SIZE_DISK"].values
table['BulgeHalfLightRadius'] = 10**gals["SIZE_BULGE"].values
table["gmagVar"] = (gals["COLOR_G_R"] + gals["COLOR_R_I"] + gals["MAG_I"]).values
table["rmagVar"] = (gals["COLOR_R_I"] + gals["MAG_I"]).values
table["imagVar"] = gals["MAG_I"].values
table["zmagVar"] = (gals["MAG_I"] - gals["COLOR_I_Z"]).values
table['disk_n'] = np.array([1.0,]*len(gals))
table['bulge_n'] = np.array([3.0,]*len(gals))
table['a_b'] = -1
table['a_d'] = 10**gals["SIZE_DISK"].values
table['a_b'] = (10**gals["SIZE_BULGE"].values) * 0.8
table['b_d'] = table["a_d"].values * 0.8
table['b_b'] = table["a_b"].values * 0.8
table['pa_disk'] = np.random.uniform(0, 360, size=len(gals))
table['pa_bulge'] = table["pa_disk"].values

table = pd.concat((bcg_table, table))

# fio.write("test_cluster_fsi_v03.fits", table.to_records(), clobber=True)